In [7]:
%display latex

def defvars(label, n):
    return list(var(label + '_%d' % i) for i in range(n))

def defpoly_from_basis(label, basis):
    coeffs = defvars(label,len(basis))
    poly = sum(c*x for c,x in zip(coeffs,basis))
    return (coeffs,poly)

def defpoly(label, d): 
    return defpoly_from_basis(label, list(x**i for i in range(d+1)))

def subst_poly(poly,table):
    return sum((poly.monomial_coefficient(v).subs(table) * v for v in poly.monomials()))

def poly_to_sym(poly):
    return sum([b*n^a for (a,b) in enumerate(poly)])

def solve_system(target, variables):
    system_vars = list(set(flatten(list(x.variables() for x in target))))
    relevant_vars = list(filter(lambda y: y in system_vars, variables))
    return solve(target, relevant_vars)

In [8]:
# GM17 (old) CRS to simplify the attempts to prove it sound without V2

n = 3
m = 6
l = 4
q = 1

var('alpha,beta,gamma,x') # CRS trapdoors
var('r') # proof random

# TODO WE DON'T GENERATE SIM INST/WITNESSES VARS
s_phi = list(var('s_%d' % i) for i in range(l+1))
s_wit = list(var('s_%d' % i) for i in range(l+1,m+1))

ringvars = ['alpha','beta','gamma','r','x']
#ringvars.extend(list(('s_%d' % i) for i in range(l+1, m+1)))
ringvars.extend(list(('mu_%d' % i) for i in range(q)))

R = LaurentPolynomialRing(SR, ringvars)
R.inject_variables()

mu = list(R('mu_%d' % i) for i in range(q))

s_all = s_phi + s_wit

t_coeffs,t = defpoly('t',n)
h_coeffs,h = defpoly('h',n-2)

us = []
uscoeffs = []
vs = []
vscoeffs = []
ws = []
wscoeffs = []

for i in range(m+1):
    u_coeffs,u = defpoly('u_%d'%i,n-1)
    us.append(u); uscoeffs.append(u_coeffs)
    
    v_coeffs,v = defpoly('v_%d'%i,n-1)
    vs.append(v); vscoeffs.append(v_coeffs)
    
    w_coeffs,w = defpoly('w_%d'%i,n-1)
    ws.append(w); wscoeffs.append(w_coeffs)

f = sum(s_all[i] * ((alpha + beta) * gamma * us[i] + gamma ** 2 * ws[i]) for i in range(0,l+1))
    
sigma_1 = [alpha,beta,gamma*t,gamma**2*t**2,(alpha+beta)*gamma*t] + \
            list(gamma * x**i for i in range(n)) + \
            list(gamma ** 2 * t * x**i for i in range(n)) + \
            list(gamma * ws[i] + (alpha+beta) * us[i] for i in range(l+1)) + \
            list(gamma ** 2 * ws[i] + gamma * (alpha+beta) * us[i] for i in range(l+1,m+1)) + \
            list(mu[i] for i in range(q)) + \
            list(mu[i] ** 2 + (alpha + beta) * mu[i] - f for i in range(q))
sigma_2 = [1,beta,gamma*t] + \
            list(gamma * x**i for i in range(n)) + \
            list(mu[i] for i in range(q))

var('A_1,A_2,A_3,A_4,A_5')
A6_vars = defvars('A_6', n)
A7_vars = defvars('A_7', n)
A8_vars = defvars('A_8', l+1)
A9_vars = defvars('A_9', m-l)
A10_vars = defvars('A_10', q)
A11_vars = defvars('A_11', q)

var('B_1,B_2,B_3')
B4_vars = defvars('B_4', n)
B5_vars = defvars('B_5', q)

var('C_1,C_2,C_3,C_4,C_5')
C6_vars = defvars('C_6', n)
C7_vars = defvars('C_7', n)
C8_vars = defvars('C_8', l+1)
C9_vars = defvars('C_9', m-l)
C10_vars = defvars('C_10', q)
C11_vars = defvars('C_11', q)

A_vars = [A_1,A_2,A_3,A_4,A_5] + A6_vars + A7_vars + A8_vars + A9_vars + A10_vars + A11_vars
B_vars = [B_1,B_2,B_3] + B4_vars + B5_vars
C_vars = [C_1,C_2,C_3,C_4,C_5] + C6_vars + C7_vars + C8_vars + C9_vars + C10_vars + C11_vars

ABC_vars = A_vars + B_vars + C_vars

A = sum(c*x for c,x in zip(A_vars,sigma_1))
B = sum(c*x for c,x in zip(B_vars,sigma_2))
C = sum(c*x for c,x in zip(C_vars,sigma_1))

V1_1_r = (A + alpha) * (B + beta) - alpha*beta - f - C


ss_phi = list(var('ss_%d' % i) for i in range(l+1))
ss_wit = list(var('ss_%d' % i) for i in range(l+1, m+1))
ss_all = ss_phi + ss_wit

qsp_eq = sum(ss_all[i] * us[i] for i in range(m+1)) * sum(ss_all[i] * us[i] for i in range(m+1)) - \
            sum(ss_all[i] * ws[i] for i in range(m+1)) - h * t

Defining alpha, beta, gamma, r, x, mu_0


In [9]:


#known_sols = dict({A_4: 0, A_5: 0},
#    **{v: 0 for v in A9_vars + A7_vars})
#eq_mod = subst_poly(V1_1_r, known_sols)

In [12]:
def print_coeff(coeff):
    show(V1_1_r.monomial_coefficient(coeff))
#    show(eq_mod.monomial_coefficient(coeff))
    print("--------------------------")

print_coeff(beta * gamma**2 * x^(2*n))
#print_coeff(alpha)
#print_coeff(alpha * x)
#print_coeff(alpha * x**2)
#print_coeff(beta)
#print_coeff(beta * x)
#print_coeff(beta * x**2)
#print_coeff(gamma**3 * x**(3*n))
#print_coeff(alpha * gamma**2 * x**(2*n))
#print_coeff(beta * gamma**2 * x**(2*n))
#print_coeff(alpha * beta)
#print_coeff(alpha * beta * x)
#print_coeff(alpha * beta * gamma)
#print_coeff(alpha*gamma)
#print_coeff(alpha ** 2)
#print_coeff(gamma * alpha * x**n)
#print_coeff(gamma * beta * x**n)
#print_coeff(gamma**2 * x**n)
#show(qsp_eq)

A_4*(B_2 + 1)*t_3^2 + A_5*B_3*t_3^2

--------------------------


In [4]:
system = V1_1_r.coefficients()
print(len(system))
sols = solve_system(system, ABC_vars + s_all)

print(len(sols))
print(sols)
with open('gm17_sols_4.txt', 'w') as file:
    file.write(dumps(sols))

67


RuntimeError: ECL says: Console interrupt.

In [5]:
with open('gm17_sols_4.txt', 'r') as file:
    content = file.read()
    sols = loads(content)
    print(len(sols))
    if (len(sols) < 5):
        for s in sols:    
            print('-----------')
            show(s)
            #show(subst_poly(A,s))
            #show(subst_poly(B,s))
            #show(subst_poly(C,s))
    else:
        print("not printing sols")

140
not printing sols


In [16]:
# Interpreting the solution
import random
import itertools

system = V1_1_r.coefficients()
totallen = len(system)
print(totallen)

allmons = V1_1_r.monomials()

#mons = [alpha * beta, alpha, beta, alpha ** 2, beta ** 2] + \
#  [alpha * beta * gamma, alpha * gamma**(n-1), beta**2 * x, beta**2 * x**(n-1), alpha * gamma * x**(2*n-1), beta * x, alpha * x**(n-1), beta * gamma * x]

mons = [] # alpha * beta, alpha, beta]
#        gamma**3 * x**(3*n), \
#        gamma**3 * x**(3*n-1), \
#        gamma**3 * x**(3*n-2), \
#        gamma**3 * x**(2*n), \
#        gamma**3 * x**(2*n-1), \
#        gamma**3 * x**(2*n-2), \
#        beta * gamma**2 * x**(2*n), \
#        beta * gamma**2 * x**(2*n-1), \
#        alpha * gamma**2 * x**(2*n), \
#        alpha * gamma**2 * x**(2*n-1), \
#        
#        #alpha * beta * x, \
#        #alpha * x, \
#        #beta ** 2 * gamma * x ** 2, \
#        #beta * x, \
#        #gamma, \
#        #beta ** 2 * gamma, \
#        
#        #beta**2 * gamma * x**n, \
#        #beta*alpha * gamma * x**n, \
#        #alpha**2 * gamma * x**n, \
#        #alpha * beta * gamma * x, \
#        #gamma ** 2, \
#        #gamma ** 3, \
#        #beta**2 * gamma, \
#        #alpha * beta * gamma, \
#        #alpha * gamma ** 2, \
#        #alpha * beta * x, \
#        #alpha * gamma, \
#        #beta * gamma, \
#        #alpha * gamma * x \
#       ]
#mons = allmons
print(len(system))

add_to_sys = [A_4, A_5] + A9_vars + A7_vars

system_0 = list(V1_1_r.monomial_coefficient(mon) for mon in mons) + add_to_sys

for i in range(30):
    sol = solve_system(system_0, ABC_vars + s_all)
    show(sol)
    solweight = len(dumps(sol))
    #print(solweight)
    
    notincludedmons = filter(lambda m: m not in mons, allmons)
    
    totry = list(itertools.combinations(notincludedmons, 1))
    random.shuffle(totry)
    #if i > 20:
    #    totry = totry[:5]
    #else:
    #    totry = totry[:50]
    
    all_sols = {}
    
    for ms in totry:
        system = system_0 + list(V1_1_r.monomial_coefficient(m) for m in ms)
        sols = solve_system(system, ABC_vars + s_all)
        if len(sols) == 0:
            print("Zero solutions achieved")
    
        #weight = max(map(lambda x: len(dumps(x)), sols))
        weight = len(dumps(sols))
        #print(weight)
        
        if weight < solweight | weight > solweight + 100:
            all_sols.update({ms:weight})
        
    nextmons = min(all_sols, key=all_sols.get) 
    system_0 = system_0 + list(V1_1_r.monomial_coefficient(m) for m in nextmons)
    mons.extend(nextmons)
    
    print("-------------------------------------------")
    print(i)
    for m in nextmons:
        show(m)
        show(V1_1_r.monomial_coefficient(m))

NameError: name 'V1_1_r' is not defined

In [41]:
var('t_n')
system = [ 
           A_3 * t_n * (B_2 + 1) + B_3 - C_5 
         , B_3 - C_5 * (B_2 + 1) 
         , B_3 * B_2 + (B_2 + 1)**2 * A_3 
         , A_3 * B_3 - C_4 
         , B_3 * (A_1 - A_2) - A_3 * (B_2 + 1)
         ]
sols = solve_system(system, ABC_vars + s_all + [t_n])

show(sols)

[[A_1 == r101, A_2 == r102, A_3 == 0, B_2 == r103, B_3 == 0, C_4 == 0, C_5 == 0, t_n == r104], [A_1 == r105, A_2 == r106, A_3 == r107, B_2 == -1, B_3 == 0, C_4 == 0, C_5 == 0, t_n == r108], [A_1 == r109, A_2 == r109, A_3 == 0, B_2 == 0, B_3 == r110, C_4 == 0, C_5 == r110, t_n == r111], [A_1 == -(2*r112 - (r113^2 - sqrt(r113^2 + 4*r114)*r113 + 2*r114)*(r112 - 1)/r114)/((r113^2 - sqrt(r113^2 + 4*r114)*r113 + 2*r114)/r114 - 2), A_2 == r112, A_3 == 2*(r113^2 - sqrt(r113^2 + 4*r114)*r113 + 2*r114)*r113/(r114*((r113^2 - sqrt(r113^2 + 4*r114)*r113 + 2*r114)^2/r114^2 - 4*(r113^2 - sqrt(r113^2 + 4*r114)*r113 + 2*r114)/r114 + 4)), B_2 == -1/2*(r113^2 - sqrt(r113^2 + 4*r114)*r113 + 2*r114)/r114, B_3 == r113, C_4 == r114, C_5 == 1/2*r113 + 1/2*sqrt(r113^2 + 4*r114), t_n == 1], [A_1 == -(2*r115 - (r116^2 + sqrt(r116^2 + 4*r117)*r116 + 2*r117)*(r115 - 1)/r117)/((r116^2 + sqrt(r116^2 + 4*r117)*r116 + 2*r117)/r117 - 2), A_2 == r115, A_3 == 2*(r116^2 + sqrt(r116^2 + 4*r117)*r116 + 2*r117)*r116/(r117*((r116^2 + sqrt(r116^2 + 4*r117)*r116 + 2*r117)^2/r117^2 - 4*(r116^2 + sqrt(r116^2 + 4*r117)*r116 + 2*r117)/r117 + 4)), B_2 == -1/2*(r116^2 + sqrt(r116^2 + 4*r117)*r116 + 2*r117)/r117, B_3 == r116, C_4 == r117, C_5 == 1/2*r116 - 1/2*sqrt(r116^2 + 4*r117), t_n == 1]]

In [6]:
#a = defvars("A",12)
#b = defvars("B",6)
#c = defvars("C",12)
#allvars = a + b + c 

system = [ 
    
           # alpha * beta
           (A_2 + A_1 + 1) * (B_2 + 1) - 1
    
          #, B_3 + (B_2 + 1) ** 2 * A_2
#
          ## \gamma
          #, A_3 * B_1 - C_3
#
          ## t(x) related
          #, A_3 * B_3 - C_4
          #, B_3 - C_5 * (B_2 + 1)
          #, A_3 * (B_2 + 1) - C_5
             
         # Four \mu related
         , A_10_0 * B_5_0 - C_11_0
          , C_11_0 - B_5_0 / (B_2 + 1)
          , C_11_0 - A_10_0 * (B_2 + 1) - B_5_0 * A_2
          , A_10_0 * B_1 - C_10_0
          
          # C_11 guess
          #, C_11 - (B_2+1) ** 2
          #, C_11 ((B_2 + 1)^2 - (A_1 + 1)*(B_2 + 1) + 1) - (B_2 + 1) ** 4

#          # Extra assumptions
        #  , B_1
        #  , C_3
        #, A_1
        , A_2
         ]
sols = solve_system(system, ABC_vars) # [A_0,B_2]) # ABC_vars + s_all + [t_n,A_10,B_5,C_10,C_11])


show(sols)

[[A_1 == -r5/(r5 + 1), A_2 == 0, A_10_0 == (1/(r5 + 1)), B_1 == (r5 + 1)*r6, B_2 == r5, B_5_0 == r5 + 1, C_10_0 == r6, C_11_0 == 1], [A_1 == -r8/(r8 + 1), A_2 == 0, A_10_0 == 0, B_1 == r7, B_2 == r8, B_5_0 == 0, C_10_0 == 0, C_11_0 == 0]]

In [13]:
curvars = var("D,T,A,B,O,N,L,G,E,R")

def toBase(vs):
    return sum(map(lambda (x,i): x * 10 ** i, zip(vs, range(0,len(vs)))))

sols = solve([toBase([D,O,N,A,L,D]) + toBase([G,E,R,A,L,D]) - toBase([R,O,B,E,R,T]), D - 5], curvars)
print(sols[0])

[D == 5, T == -9901/100000*r25 - 99/10000*r26 + 1/100000*r27 + 1/5*r28 + 1/1000*r29 - 1/1000*r31 + 1/50*r32 + 200001/20000, A == r32, B == r31, O == r30, N == r29, L == r28, G == r27, E == r26, R == r25]


In [32]:
abvars = var("B_2,A_1")

sols = solve([(B_2+1)**2 - (A_1+1)*(B_2+1) + 1],abvars)
print(sols)

[
[B_2 == 1/2*r183 + 1/2*sqrt(r183^2 + 2*r183 - 3) - 1/2, A_1 == r183],
[B_2 == 1/2*r184 - 1/2*sqrt(r184^2 + 2*r184 - 3) - 1/2, A_1 == r184]
]


In [13]:
var('alpha,beta,gamma,t,mu,r,z')

A = (r-1)*beta + (1-r)*gamma*t + r * mu
B = (1 - r) / r * beta + (1 - r) / (r**2) * gamma * t + mu
C = (r-1)**2 / (r**2) * gamma**2 * t**2 + (1 - r) / r * (alpha + beta) * gamma * t + r (mu**2 + (alpha + beta) * mu + z)

show((A + alpha) * (B + beta) - alpha * beta - z - C)

-gamma^2*(r - 1)^2*t^2/r^2 + (alpha + beta)*gamma*(r - 1)*t/r - (gamma*(r - 1)*t - beta*(r - 1) - mu*r - alpha)*(beta + mu - beta*(r - 1)/r - gamma*(r - 1)*t/r^2) - alpha*beta - (alpha + beta)*mu - mu^2 - 2*z

In [ ]:
var('gamma,t,r_1,r_2,alpha,beta,x,w_1,w_2,f_1,f_2')